In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import math
import random
import numpy as np
import tensorflow as tf

In [2]:
from nets import nets_factory
from preprocessing import preprocessing_factory
slim = tf.contrib.slim

In [ ]:
num_classes = 340
batch_size = 256
image_size = 128
num_samples_per_epoch = 

In [3]:
def decode(serialized_example, frame_or_flow='flow'):
    features = tf.parse_single_example(serialized_example,features={
        'label':tf.FixedLenFeature([], tf.int64),
        'image_raw':tf.FixedLenFeature([], tf.string)
        })
    image = tf.image.decode_png(features['image_raw'])
    label = tf.cast(features['label'], tf.int32)

    image = tf.reshape(image, [image_size, image_size, 1])
    
    return image, label

feat_dir = '/home/forwchen/daily/190114/tfrecords'
files = tf.data.Dataset.list_files(feat_dir+'/val-*.tfrecord').shuffle(300, seed=1234)

ds = files.apply(tf.contrib.data.parallel_interleave(
        tf.data.TFRecordDataset, cycle_length=8))
ds = ds.map(decode, num_parallel_calls=16)
ds = ds.apply(tf.contrib.data.shuffle_and_repeat(buffer_size=batch_size*10, seed=1234))
ds = ds.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
ds = ds.prefetch(buffer_size=batch_size * 4)

iterator = tf.data.Iterator.from_structure(ds.output_types,
                                           ds.output_shapes)
images, labels = iterator.get_next()

training_init_op = iterator.make_initializer(ds)

Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use `tf.data.experimental.shuffle_and_repeat(...)`.
Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.


In [4]:
image_prep_fn = preprocessing_factory.get_preprocessing('inception_v1', is_training=False)
images_preped = image_prep_fn(images, None, None)
print images, images_preped

net_fn = nets_factory.get_network_fn('inception_v1', num_classes=None, is_training=True)
logits, end_points = net_fn(images_preped)
print logits

logits = tf.squeeze(logits)
class_logits = tf.layers.dense(logits, num_classes)

labels_oh = tf.one_hot(labels, num_classes, on_value=1., off_value=0., dtype=tf.float32)

loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels_oh, logits=class_logits)
loss = tf.reduce_mean(loss)

inception_prep no aug
Tensor("IteratorGetNext:0", shape=(128, 128, 128, 1), dtype=uint8) Tensor("eval_image/Mul:0", shape=(128, 128, 128, 1), dtype=float32)
INFO:tensorflow:Scale of 0 disables regularizer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Tensor("InceptionV1/Logits/global_pool:0", shape=(128, 1, 1, 1024), dtype=float32)


In [ ]:
global_step = tf.Variable(0, trainable=False, name='global_step')



In [5]:
#sess = tf.Session()
#sess.run(training_init_op)